# CatBoost (Gradient-boosting), performance between customer order quantities

In [1]:
%pip install nbformat pandas numpy matplotlib seaborn scikit-learn catboost 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
import sys
sys.path.append('../../src')

In [4]:
from data_config import ProductHandler as ph

d:\SCM---ML-Visualisation\notebooks\catboost\../../src\datasets.py:13: DtypeWarning: Columns (4,6,20,50,52) have mixed types. Specify dtype option on import or set low_memory=False.
  products_s = pd.read_csv(
d:\SCM---ML-Visualisation\notebooks\catboost\../../src\datasets.py:15: DtypeWarning: Columns (5,29,42,48) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_i = pd.read_csv(
d:\SCM---ML-Visualisation\notebooks\catboost\../../src\datasets.py:18: DtypeWarning: Columns (43,57,106,141,146,211,212,247,255,294,298,316,327,332,350,354) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_m = pd.read_csv(
d:\SCM---ML-Visualisation\notebooks\catboost\../../src\datasets.py:21: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  sp_inv_adds = pd.read_csv(


----- DROP BOOLEAN AND NULL COLUMNS (NOT NEEDED) -----

Merged Data - Columns to be dropped (Missing Vals): ['BackStage_DefaultReportForm', 'BackStage_SmartMarkSet', 'BillCounty', 'CoreType', 'EndUserNum', 'EndUserPO', 'Equip3_ID', 'Equip3_Status', 'Est_v_Act_Notes', 'Ink_Status', 'JDF_Note_to_DFE', 'JDF_Send_Msg', 'MiscChargeDesc1', 'MiscChargeDesc2', 'MiscChargeDesc3', 'MiscChargeDesc4', 'PlateStat', 'ProofStat', 'Roto_CEL_Product_ID', 'Roto_Quote_Line_ID', 'Roto_Quote_Number', 'Schedule_Status', 'ShipAttn_EmailAddress', 'ShipCounty', 'ShipStat', 'Tag_x', 'Tool3Descr', 'Tool4Descr', 'Tool5Descr', 'ToolNo3', 'ToolNo4', 'ToolNo5', 'ToolStat', 'UserDef_MR_1_Lb', 'UserDef_MR_2_Lb', 'Assigned', 'Desc2_x', 'ediLineNumber', 'eTraxx_Customer_Notes', 'Location_x', 'PO_Number', 'Adhesive', 'Alternate', 'eTraxx_Forecast_Range', 'Material', 'Tag_y', 'UPC']

Merged Data - Columns to be dropped (All cols with rows same vals): ['Act_OTHER_Hours', 'ActArtwork', 'ActFinMaterial', 'ActPostPressHours',

d:\SCM---ML-Visualisation\notebooks\catboost\../../src\datasets.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_data['OrderDate'] = pd.to_datetime(
d:\SCM---ML-Visualisation\notebooks\catboost\../../src\datasets.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['OrderDate'] = pd.to_datetime(
d:\SCM---ML-Visualisation\notebooks\catboost\../../src\datasets.py:110: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_data['Ship_by_Date'] = pd.to_

In [5]:
ph.custom_prod_set()

All custom codes for existing products: 
{'570', '12U', '693', '694', 'JF-', '73T', '637', 'OCI', 'GOO', '171', '58Z', '712', '675', '322', '409', '13U', '676', '19T', '051', '639', '57Z', '394', '054', '92U', '129', '05V', '06T', '98X', 'JMP', '755', '786', 'PRA', '654', '812', '13Z', '33U', '483', '985', 'VM-', '74Y', 'RS-', 'FTM', 'TH-', '40Z', '10U', '188', '41X', '57X', '56X', '531', '477', '00Z', '91Y', '395', '874', '714', 'SBH', 'GK-', '24V', '257', '365', '164', '49Z', '81Y', '385', 'OY-', 'PIE', '481', '593', 'GRE', '528', 'T-0', '24Y', '699', '042', 'GOL', '07X', '89V', 'SK-', '555', '951', '3-0', '31W', '749', '937', '40U', '469', '566', 'PLZ', 'HOU', '220', '82Y', '101', '65V', '991', '120', '353', '565', '86X', '12X', '520', '631', '21X', '460', '05X', '291', '06X', 'DMO', 'RAV', '62T', '809', 'RMF', '488', '397', '829', '32X', 'CBO', '300', '05Z', 'VSE', '59Z', '32T', '00z', '47Y', '771', '11W', 'N-0', '375', 'KIM', '198', '823', 'EL-', 'ON-', '334', 'U-T', 'CMI', '507',

In [6]:
# define the features and target variable from 'product_sales'
# CHANGE THE CUSTOMER CODE HERE >>>>>>>>>>
custom_code_df = pd.DataFrame(ph.get_custom_code_data('FRE'))

# customer code for later referencing
custom_ref = custom_code_df.loc[0, 'ProductNumber'][:3].lower().upper()
print(custom_ref)

custom_code_df.head()

FRE


,ProductNumber,order_year,order_month,order_week,order_weekday,is_weekend,OrderDate,Customer_Num,OrderQuantity,prev_month_sales,...,moving_avg_3m,moving_avg_6m,moving_avg_12m,moving_avg_18m,sales_2022,sales_2023,sales_2024,growth_2023,growth_2024,PhysicalInv
0,FRE-005-0006Y,2022,1,3,1,0,2022-01-18,66.0,40000,80000.00000,...,53333.333333,61000.000000,61000.000000,61000.000000,40000,0,0,-100.0,NaN,0
1,FRE-005-0006Y,2022,3,10,2,0,2022-03-09,66.0,80000,40000.00000,...,50000.000000,66250.000000,66250.000000,66250.000000,80000,0,0,-100.0,NaN,0
2,FRE-005-0006Y,2022,4,14,3,0,2022-04-07,66.0,40000,30000.00000,...,61666.666667,61666.666667,61666.666667,61666.666667,40000,0,0,-100.0,NaN,0
3,FRE-005-0006Y,2022,6,24,1,0,2022-06-14,66.0,30000,115000.00000,...,72500.000000,72500.000000,72500.000000,72500.000000,30000,0,0,-100.0,NaN,0
4,FRE-005-0006Y,2022,7,30,1,0,2022-07-26,66.0,115000,110031.53337,...,115000.000000,115000.000000,115000.000000,115000.000000,115000,0,0,-100.0,NaN,0


In [7]:
from app_utils import Transform as trans
from model_utils import *
from sklearn.pipeline import Pipeline

d:\SCM---ML-Visualisation\tf_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Compute OrderQuantity values with a zscore with threshold of 3 and remove them (Outlier removal)
custom_code_df = trans.compute_zscore(custom_code_df)

custom_code_df.head()

d:\SCM---ML-Visualisation\notebooks\catboost\../../src\app_utils.py:27: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  group['z_score'] = np.abs(stats.zscore(group['OrderQuantity']))
d:\SCM---ML-Visualisation\notebooks\catboost\../../src\app_utils.py:27: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  group['z_score'] = np.abs(stats.zscore(group['OrderQuantity']))
d:\SCM---ML-Visualisation\notebooks\catboost\../../src\app_utils.py:27: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  group['z_score'] = np.abs(stats.zscore(group['OrderQuantity']))
d:\SCM---ML-Visualisation\notebooks\catboost\../../src\app_utils.py:27: R

ProductNumber  order_year  order_month  order_week  \
ProductNumber                                                         
FRE-005-0006Y 0  FRE-005-0006Y        2022            1           3   
              1  FRE-005-0006Y        2022            3          10   
              2  FRE-005-0006Y        2022            4          14   
              3  FRE-005-0006Y        2022            6          24   
              4  FRE-005-0006Y        2022            7          30   

                 order_weekday  is_weekend  OrderDate  Customer_Num  \
ProductNumber                                                         
FRE-005-0006Y 0              1           0 2022-01-18          66.0   
              1              2           0 2022-03-09          66.0   
              2              3           0 2022-04-07          66.0   
              3              1           0 2022-06-14          66.0   
              4              1           0 2022-07-26          66.0   

                 OrderQuantity  prev_month_sales  ...  moving_avg_6m  \
ProductNumber                                     ...                  
FRE-005-0006Y 0          40000       80000.00000  ...   61000.000000   
              1          80000       40000.00000  ...   66250.000000   
              2          40000       30000.00000  ...   61666.666667   
              3          30000      115000.00000  ...   72500.000000   
              4         115000      110031.53337  ...  115000.000000   

                 moving_avg_12m  moving_avg_18m  sales_2022  sales_2023  \
ProductNumber                                                             
FRE-005-0006Y 0    61000.000000    61000.000000       40000           0   
              1    66250.000000    66250.000000       80000           0   
              2    61666.666667    61666.666667       40000           0   
              3    72500.000000    72500.000000       30000           0   
              4   115000.000000   115000.000000      115000           0   

                 sales_2024  growth_2023  growth_2024  PhysicalInv  z_score  
ProductNumber                                                                
FRE-005-0006Y 0           0       -100.0          NaN            0  0.65625  
              1           0       -100.0          NaN            0  0.59375  
              2           0       -100.0          NaN            0  0.65625  
              3           0       -100.0          NaN            0  0.96875  
              4           0       -100.0          NaN            0  1.68750  

[5 rows x 35 columns]

In [9]:
X = custom_code_df.drop('OrderQuantity', axis=1)
y = custom_code_df.OrderQuantity

In [10]:
# Trian and transforms data (CatBoost transforms natively so might not need to transform)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train_preprocessed, X_val_preprocessed = trans.transform_data(X_train, X_val, X)

In [11]:
from catboost import CatBoostRegressor

In [12]:
catboost_model = CatBoostRegressor(
    cat_features=[ # Specify catagorical features for cat model, else model fails to compute
        'ProductNumber',
        # 'order_month', 
        # 'order_week', 
        # 'order_weekday'
        ],
    silent=True # Disable for training logs
)

In [13]:
catb_params = find_best_hyperparameters(catboost_model, param_grids(CatBoostRegressor().__class__.__name__), X_train, y_train)

Model type: CatBoostRegressor
Fitting 5 folds for each of 256 candidates, totalling 1280 fits
CatBoostRegressor Best Parameters: {'colsample_bylevel': 0.8, 'depth': 6, 'grow_policy': 'Depthwise', 'iterations': 1000, 'l2_leaf_reg': 3, 'learning_rate': 0.03, 'min_data_in_leaf': 5, 'random_state': 42, 'subsample': 1.0}


In [14]:
# Train and get metric scores of CatBoost with best parameters
catb_params = CatBoostRegressor(**catb_params)

catb_params.fit(X_train, y_train)

print('----------- TRAINING METRICS -----------')
train_metrics = evaluate_model(catb_params, X_train, y_train)
print('\n\n')
print('----------- TESTING METRICS -----------')
test_metrics = evaluate_model(catb_params, X_val, y_val)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="FRE-005-0344Z": Cannot convert 'b'FRE-005-0344Z'' to float

In [ ]:
# Get model predictions
y_train_pred = catb_params.predict(X_train)
y_val_pred = catb_params.predict(X_val)

In [ ]:
# Create a DetailedOrderDate column using week and weekday information.
custom_code_df['DetailedOrderDate'] = pd.to_datetime(
    custom_code_df['order_year'].astype(str) +
    custom_code_df['order_week'].astype(str).str.zfill(2) +
    custom_code_df['order_weekday'].astype(str),
    format='%Y%W%w'
)

In [ ]:
# Create subplots (2 rows, 2 columns: one for line plot, one for residual plot)
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18, 12)) 
fig.subplots_adjust(hspace=0.5, wspace=0.4)

# Time Series Comparison
sns.lineplot(x=custom_code_df.loc[y_val.index, 'OrderDate'], y=y_val, label='Actual', ax=axes[0, 0])
sns.lineplot(x=custom_code_df.loc[y_val.index, 'OrderDate'], y=y_val_pred, label='Predicted', ax=axes[0, 0])
axes[0, 0].set_title(f'{custom_ref} Neural Network - Time Series', fontsize=16)
axes[0, 0].set_xlabel('Date', fontsize=14)  
axes[0, 0].set_ylabel('Order Quantity', fontsize=14)

# Residual Plot
residuals = y_val - y_val_pred
sns.scatterplot(x=y_val_pred, y=residuals, alpha=0.6, ax=axes[0, 1])
axes[0, 1].axhline(0, color='r', linestyle='--')
axes[0, 1].set_title(f'{custom_ref} Neural Network - Residuals', fontsize=16)  
axes[0, 1].set_xlabel('Predicted Values', fontsize=14)  
axes[0, 1].set_ylabel('Scaled Residuals', fontsize=14)

# Actual vs Predicted Scatter Plot
min_val = min(y_val.min(), y_val_pred.min())
max_val = max(y_val.max(), y_val_pred.max())
sns.scatterplot(x=y_val, y=y_val_pred, alpha=0.6, ax=axes[1, 0], label='Predicted')
axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=1)  # Reference line
axes[1, 0].set_title(f'{custom_ref} Neural Network - Accuracy', fontsize=16)
axes[1, 0].set_xlabel('Actual Values', fontsize=14)
axes[1, 0].set_ylabel('Predicted Values', fontsize=14)  
axes[1, 0].legend(fontsize=12)

# Monthly Trend Comparison
monthly_data = custom_code_df[['order_month']].loc[custom_code_df.index.intersection(X_val.index)].copy()
monthly_data['Actual'] = y_val
monthly_data['Predicted'] = y_val_pred

sns.lineplot(x='order_month', y='Predicted', data=monthly_data, label='Predicted', ax=axes[1, 1])
sns.lineplot(x='order_month', y='Actual', data=monthly_data, label='Actual', ax=axes[1, 1], color='black', linestyle='--')
axes[1, 1].set_title(f'{custom_ref} Neural Network - Monthly Trend Comparison (2022-2025)', fontsize=16)  
axes[1, 1].set_xlabel('Month', fontsize=14)
axes[1, 1].set_ylabel('Order Quantity', fontsize=14)

# Adjust x-tick labels
plt.setp(axes[0, 0].get_xticklabels(), rotation=45, ha='right', fontsize=12)
plt.setp(axes[1, 1].get_xticklabels(), rotation=45, ha='right', fontsize=12)

plt.tight_layout()
plt.show()